In [1]:
import numpy as np

# 1. Download the Dataset and Understand the Format

# 2. Generate the Data Matrix and the Label vector

# 3. Split the Dataset into Training and Test sets

# 4. Classification using PCA

# 5. Classification Using LDA 

In [ ]:
center_function = lambda x: x - x.mean()

In [ ]:
def lda(data):
  # Lenght of sample in each class
  l = 5
  # Calculate the mean of feature for every class
  # Mean matrix is 10304x1 dim
  mean_k = [[1, 2, 3], [11, 22, 33], [10, 20, 30]]
  mean = [5,6,7]
  # Calculate between-class scatter matrix
  # B matrix is 10304 x 10304
  bmat = []
  for i in range(len(mean_k)):
    bmat.append(np.multiply(l, np.dot(np.subtract(mean_k[i], mean), np.subtract(mean_k[i], mean).T)))
  bmat = np.sum(bmat, axis=0)
  # Center class matrix 
  # Zi matrix is 5 x 10304
  zmat = center_function(data)
  
  # Class scatter matrix 
  # S matrix is 10304 x 10304
  smat = []
  for si in zmat:
    smat.append(np.dot(si, si.T))

  # Calculate wthin-class scatter matrix
  s_tot = np.sum(smat, axis=0)
  
  # Compute dominant eigenvector
  eval, evec = np.linalg.eigh(np.dot(np.inv(s_tot), bmat))

  # Return the 39 dominant eigenvectors and the mean

# 6. Classifier Tuning 

# 7. Compare vs Non-Face Images

# 8. Bonus 